In [1]:
## Import relevant packages
import torch
import pandas
import spacy
import os

In [2]:
# x_train = pandas.read_csv('./data/train/train.data.txt', delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
# y_train = pandas.read_csv('./data/train/train.gold.txt', header=None, names=['label'])
# trainset = pandas.concat((x_train, y_train), axis=1)
# print(trainset.POS.unique(), trainset.label.unique())
# trainset

In [3]:
# x_val = pandas.read_csv('./data/validation/validation.data.txt', delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
# y_val = pandas.read_csv('./data/validation/validation.gold.txt', header=None, names=['label'])
# valset = pandas.concat((x_val, y_val), axis=1)
# print(valset.POS.unique(), valset.label.unique())
# valset

In [4]:
# from torchtext.vocab import build_vocab_from_iterator
# from torchtext.data.utils import get_tokenizer

# tokenizer = get_tokenizer('basic_english')
# def _build_vocab(corpus):
#     for sen in corpus:
#         yield tokenize(sen)
        
# spl_tokens = ["<UNK>", "<SEP>", "<PAD>"]
# UNK_id, SEP_id, PAD_id = 0, 1, 2

# corpus_iter = trainset.loc[:,['word','sen1','sen2']].values.reshape(-1)
# vocab = build_vocab_from_iterator(_build_vocab(corpus_iter), specials=spl_tokens, special_first=True)
# vocab.set_default_index(UNK_id)

In [5]:
# # collate function is used when we have map-style dataset like raw data per se
# def collate_fn(dataset):
    

In [6]:
# from torch.utils.data import DataLoader
# traindata_loader = DataLoader(trainset.values, shuffle=True, batchsize=32, collate_fn=collate_fn)
# valdata_loader = DataLoader(valset.values, shuffle=False, batchsize=32, collate_fn=collate_fn)

In [7]:
# trainset.values[0]

In [8]:
# import os
# def combine_xy(path, train=True):
#     train = 'train' if train else 'validation'
#     x = open(os.path.join(path, '%s/%s.data.txt'%(train,train))).readlines()
#     y = open(os.path.join(path, '%s/%s.gold.txt'%(train,train))).readlines()
#     data = []
#     for x_,y_ in zip(x,y):
#         data.append(x_[:-1] + "\t" + y_)
#     out = open(os.path.join(path, '%s/%s.tsv'%(train,train)), 'w')
#     out.writelines(data)

In [9]:
# combine_xy('data',True)
# combine_xy('data',False)

In [10]:
# t = get_tokenizer("basic_english")
# print(t("dont"))
# import spacy
# en = spacy.load('en')
# t = en.tokenizer
# for x in t("shubham mittal"):
#     print(x.text)
# for x in t("dont"):
#     print(x.text)
# for x in t("don't"):
#     print(x.text)

In [11]:
def process_data(path, train=True):
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

In [12]:
trainset = process_data('./data', train=True)
# trainset

In [13]:
# pandas.read_csv('./data/train/train.csv', header=None)

In [14]:
valset = process_data("./data", train=False)
# valset

In [15]:
# t = get_tokenizer("basic_english")
# t(valset.loc[635,'sen2'])

In [16]:
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

In [18]:
TEXT = data.Field(
    sequential=True,
    lower=True,
    tokenize=get_tokenizer("basic_english"),
)
fields = [
    ('word', TEXT),
    ('POS', data.Field(use_vocab=False, sequential=False)), 
    ('sen1', TEXT),
    ('word1', data.Field(use_vocab=False, sequential=False)),
    ('sen2', TEXT),
    ('word2', data.Field(use_vocab=False, sequential=False)),
    ('label', data.Field(use_vocab=False, sequential=False)),
]

In [19]:
train_set, val_set = data.TabularDataset.splits(
    path = './data/',
    train = 'train/train.csv',
    validation = 'validation/validation.csv',
    format = 'csv',
    fields = fields,
    skip_header = False
)
# test_set = data.TabularDataset(
#     path='',
#     fields = fields[:-1]
#     skip_header = False,
# )
TEXT.build_vocab(train_set, vectors='glove.6B.100d')

In [33]:
print(train_set[0].word, train_set[0].sen1, train_set[0].sen2)

['carry'] ['you', 'must', 'carry', 'your', 'camping', 'gear', '.'] ['sound', 'carries', 'well', 'over', 'water', '.']


In [51]:
# print(TEXT.vocab.itos)

In [52]:
# print(vars(train_set[0]))

In [20]:
train_itr, val_itr = data.BucketIterator.splits(
    (train_set, val_set),
    sort_key = lambda sample : len(sample.sen1),
    sort = True,
    batch_size = 32,
    repeat = False
)

In [53]:
# batch = next(iter(train_itr))
# print(batch.sen1.permute(1,0)[:2])
# print(TEXT.vocab.itos[batch.sen1.permute(1,0)])

In [54]:
# for batch in val_itr:
#     print(batch)

In [21]:
# from torchtext.vocab import GloVe
# glove_emb = GloVe(name='6B', dim=100)

In [48]:
print("vocab size", len(TEXT.vocab))
# print(TEXT.vocab.stoi['carry'], TEXT.vocab.stoi['carries'], TEXT.vocab.stoi['carried'])
# print(TEXT.vocab.vectors[84,:5])
# print(TEXT.vocab.vectors[780,:5])

vocab size 7432
